In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

from sklearn.cluster import KMeans 
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering

In [3]:
import sys

sys.path.append('../../src')



In [4]:
from base_exp_gen import generate_experiment as GEN_EXP
from clustering_algos import agglomerative_clustering, kmeans_clustering, box_clustering
from sklearn.model_selection import train_test_split as datasplit
from sklearn.ensemble import GradientBoostingRegressor as GDBR
from evaluations import computeATT_per_cluster, predict_cf, calculate_ite, get_homogeneity



In [5]:
np.random.seed(seed = 0)

In [6]:
pd.set_option("display.max_columns", None)


In [7]:
x_partition = [[i,i**2,2*i**4, 50000] for i in [2,4,5,10,12,20,25]]
def its(x):
    for i, (x1,x2,x3,x4) in enumerate(x[1:]):
        x[i+1][3]=(int(x[i][3]/(2*np.sqrt(x3/x[i][2]))))
    return x

In [8]:
a = its(x_partition)
exps = pd.DataFrame(a, columns = ['X-parts','Clusters', 'N', 'iters'])

In [9]:
name = 'agglo'
exps.index.name = name

In [10]:
exps['mean-hom'] = 0
exps['std-hom'] = 0

In [11]:
exps

,X-parts,Clusters,N,iters,mean-hom,std-hom
agglo,,,,,,
0,2,4,16,100000,0,0
1,4,16,256,12500,0,0
2,5,25,625,4000,0,0
3,10,100,10000,500,0,0
4,12,144,20736,173,0,0
5,20,400,160000,31,0,0
6,25,625,390625,9,0,0


In [12]:
import time

In [13]:
%%time
col_select = ['x0', 'x1']
gen = []
for i,row in exps.iterrows():
    c, n, iters = row.Clusters, row.N, row.iters
    homc = []
    print('running:',c,n)
    ss = time.time()
    for j in range(iters):
        ###gneerate data
        N,D = n,2
        kw_generate =  {'N':n, 'D':D,'f_gen_name': 'uniform_gen','low':0, 'high': 1}
        kw_cluster =  {'f_class_name':'circle_class', 'centers': [0.5, 0.5], 'r_small': 0.2, 'r_big':0.4}
        kw_treatment = {'f_treat_name' :'uniform_treat','choices':[0,1], 'probabilities':[0.5,0.5]}

        stats = np.array([[1, 0.75], [0, 0.75], [1, 0.75], [1, 0.75]])
        kw_outcome = {'f_outcome_name': 'outcome1','treatment':'Treatment', 'cls':'C', 'stats':stats}

        data = GEN_EXP(kw_generate, kw_cluster, kw_treatment, kw_outcome).dat
        data = calculate_ite(data.copy(), treatment = 'Treatment',
                     counterfactual = 'Ycf', outcome = 'Y', ite_name = 'ITE')
        
        ### cluster data
        data2,_ = agglomerative_clustering(data, clusters=c,col_select = col_select, cluster_name = 'A')
        ### calc hom
        res = computeATT_per_cluster(data2.copy(), hom_name = 'HOM',
                                     weight_names = 'W', cluster_name  = "A", att = False)
        
        hom = (res['HOM']*res['W']).sum(axis = 0)
        homc.append(hom)
        
    ###get stats
    ee = time.time() - ss
    print('time:',ee)
    homcnp = np.array(homc)
    gen.append(homcnp)
    mean = np.mean(homcnp)
    std = np.std(homcnp)
    exps.loc[i,'mean-hom'] = mean
    exps.loc[i,'std-hom'] = std
    
    

running: 4 16
time: 3723.888333082199
running: 16 256
time: 848.3300819396973
running: 25 625
time: 393.70612931251526
running: 100 10000
time: 1748.5425987243652
running: 144 20736
time: 2279.515336036682
running: 400 160000


MemoryError: Unable to allocate 11.9 GiB for an array with shape (12799920000,) and data type bool

In [14]:
exps

,X-parts,Clusters,N,iters,mean-hom,std-hom
agglo,,,,,,
0,2,4,16,100000,0.724786,0.078580
1,4,16,256,12500,0.769803,0.036463
2,5,25,625,4000,0.812196,0.029331
3,10,100,10000,500,0.907393,0.008423
4,12,144,20736,173,0.920111,0.006705
5,20,400,160000,31,0.000000,0.000000
6,25,625,390625,9,0.000000,0.000000


In [15]:
from pathlib import Path



In [16]:
p = './homog/'
Path('./homog/').mkdir(parents = True, exist_ok = True)
exps.to_csv(p+name+'.csv')